# Dashboard application code

In [1692]:
import pandas as pd
import numpy as np
import datetime
import calendar
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input , Output
import dash_table as dt
import plotly.graph_objs as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [1693]:
data = pd.read_excel('Liste des Consultations 2021-10-08.xlsx')
data_ei = pd.read_excel("evenement_indesirable_2021-08-30.xlsx")
data_dcf_ked = pd.read_excel('Tracker_Kedougou.xlsx')
data_dcf_tam = pd.read_excel('Tracker_Tamba.xlsx')
data_dcf_mb = pd.read_excel('Tracker_Mbour.xlsx')
data.head(2)

,number,caseid,name,ADENO_CERVIC,AGE_GEST,AGITATION,AMYGDAL,ANEMIE,ANGINE,ANGINE_AUTRE_TTT,...,indices.patient,closed,closed_by_username,closed_date,last_modified_by_user_username,last_modified_date,opened_by_username,opened_date,owner_name,case_link
0,0,3e05a021-8c53-40f0-b188-573a1063cfcf,003-003-03-21-31-0001,---,NaN,---,---,N,---,---,...,1e64deaa-0a0a-440d-9a7e-a5f5a3f08beb,False,---,---,psd_sngom,2021-05-05 10:57:44,psd_sngom,2021-05-05 10:47:05,poste de santé de Dalaba,https://cchq.sn.amr.health/a/cdt/reports/case_...
1,1,1987e83f-ca5a-42a7-8da0-f4a570b7f4da,002-002-02-21-21-0003,---,NaN,---,---,N,---,---,...,9701c4d5-ea72-465b-b36b-c41e5cac6ca7,False,---,---,psnn_tdiome,2021-05-06 20:53:21,psnn_tdiome,2021-05-06 20:36:08,poste de santé de Ndiaye-Ndiaye,https://cchq.sn.amr.health/a/cdt/reports/case_...


In [1694]:
data_dcf = pd.concat([data_dcf_ked,data_dcf_mb,data_dcf_tam],axis=0)
data_dcf.head(5)

,ID patient,Site,Nom formulaire,Nom variable,Valeur à vérifier,Message de la query,Nouvelle valeur,Commentaires cliniciens
0,003-003-03-21-31-0002,Kedougou,NaN,Type de visite,NaN,Ce patient n'a pas effectué la visite D3,D3,confirmé
1,003-003-03-21-31-0002,Kedougou,Suivi de consultation J7,Etude compléte,NaN,Si le formulaire de J7 est remplie alors le fo...,Y,donnée corrigée
2,003-003-03-21-31-0003,Kedougou,NaN,Type de visite,NaN,Ce patient n'a pas effectué la visite D3,D3,confirmé
3,003-003-03-21-31-0003,Kedougou,Suivi de consultation J7,Etude compléte,NaN,Si le formulaire de J7 est remplie alors le fo...,Y,confirmé
4,003-003-03-21-32-0001,Kedougou,Suivi de consultation D3,Sifflement,NaN,La présence de sifflement doit être renseignée,non,confirmé


In [1695]:
data_dcf['Commentaires cliniciens'].unique()

array(['confirmé', 'donnée corrigée', ' confirmé', '  confirmé',
       'en cours de vérification', 'confirmé ', nan, 'confimé',
       'donneé corrigée', 'donnéé corrigée', 'donnée corriger',
       'donnée confirmée', 'donnée conforme', 'oui donné conforme',
       'Donnée corrigée', 'à vérifier'], dtype=object)

In [1696]:
liste1 = [' confirmé','  confirmé','confirmé ','confimé','confirmé ','confimé',
                                             'donneé corrigée','donnéé corrigée','donnée corriger','donnée confirmée',
                                                  'donnée conforme','oui donné conforme','Donnée corrigée','à vérifier']

liste2 = ['confirmé','confirmé','confirmé','confirmé',
                                             'confirmé','confirmé','donnée corrigée','donnée corrigée','donnée corrigée',
                                            'confirmé','confirmé','confirmé','donnée corrigée','en cours de vérification'
                                            ]


data_dcf['Commentaires cliniciens'].replace(liste1,liste2,inplace=True)
data_dcf['Commentaires cliniciens'].unique()

array(['confirmé', 'donnée corrigée', 'en cours de vérification', nan],
      dtype=object)

In [1697]:
data.TYPE_VIS.unique()

array(['---', 'D3', 'D7', 'D0', 'UV'], dtype=object)

In [1698]:
data = data[(data["ARM"] == "CDS") | (data["ARM"] == "CONTROL")]
data.ARM.unique()

data.CENTRE.replace([1,2,3,4],["Mbour","Fatick","Kedougou","Tambacounda"],inplace=True)
data['SEXE'].replace(['0','1'],['Female','Male'],inplace=True)
data.STUDY_COMPLETE.replace(['Y','N','---'],['Yes','No','NA'],inplace=True)
data.FIEVRE_YN.replace(['1','0'],['Fever','No fever'],inplace=True)
data.DIARRHEES.replace(['Y','N'],['Diarrhea','No diarrhea'],inplace=True)
data.ARBOVIROSE.replace(['Y','N'],['Yes','No'],inplace=True)
data.MGORGE.replace(['Y','N'],['Yes','No'],inplace=True)
data.TDR_DENGUE_FAIT.replace(['Y','N'],['Yes','No'],inplace=True)
data.TDR_STREPTO_A_FAIT.replace(['Y','N'],['Yes','No'],inplace=True)
data.TTT_ATBT_PRESCRIT.replace(['Y','N'],['Yes','No'],inplace=True)
data.SELLES.replace(['Y','N'],['Yes','No'],inplace=True)
data.SANGUIN.replace(['Y','N'],['Yes','No'],inplace=True)
data.ECOUV_NASAL.replace(['Y','N'],['Yes','No'],inplace=True)
data.SYND_GRIPPAL.replace(['Y','N'],['Yes','No'],inplace=True)
data.TTT_ATBT_PRESCRIT.replace(['Y','arret_traitement'],['Yes','No'],inplace=True)

data.DAT_CST = pd.to_datetime(data.DAT_CST, utc=False)

# data.AGE= data.AGE.astype(float)
# data.AGE_MOIS= data.AGE_MOIS.astype(float)
data.AGE.replace('---',np.nan,inplace=True)
data.AGE_MOIS.replace('---',np.nan,inplace=True)

# data_inc.shape

data.AGE = data.AGE.astype(float)
data.AGE_MOIS = data.AGE_MOIS.astype(float)
data.reset_index(inplace=True,drop=True)

for i in range(data.shape[0]):
    if data.AGE[i] == 0:
        data.AGE.replace(data.AGE[i],data.AGE_MOIS[i]/12,inplace=True)
        
bins = [0.5,4,9,15]
groupe_names = ['[6mois-5ans[','[5ans-10ans[','[10ans-15ans[']
data['AGE_CAT'] = pd.cut(data['AGE'],bins,labels=groupe_names,include_lowest=True,duplicates='drop')

# -------------------------------------------------------------------------------------------------------------------------

data_inc = data[data.TYPE_VIS == 'D0']
data_inc.reset_index(inplace = True, drop = True)

data_d3 = data[data.TYPE_VIS == 'D3']
data_d3.reset_index(inplace = True, drop = True)

data_d7 = data[data.TYPE_VIS == 'D7']
data_d7.reset_index(inplace = True, drop = True)

data_uv = data[data.TYPE_VIS == 'UV']
data_uv.reset_index(inplace = True, drop = True)

data_nan = data[data.TYPE_VIS == '---']
data_nan.reset_index(inplace = True, drop = True)

df = pd.read_excel('Classeur8.xlsx')
df['Ln']  = -1*df['Ln']

data_for_map = data_inc.groupby('CENTRE').count()['ARM'].to_frame()
data_for_map.reset_index(inplace=True)
data_for_map
df['Nbre_inclus'] = data_for_map.ARM.values

In [1741]:
# check = data_inc[data_inc.DAT_CST >= pd.to_datetime('2021-09-13')][['name','DAT_CST','registration_time']]
# check
print(data[data.TYPE_VIS == 'D0'].shape)
print(data[data.TYPE_VIS == 'D3'].shape)
print(data[data.TYPE_VIS == 'D7'].shape)


(475, 923)
(347, 923)
(301, 923)


In [1700]:
data_inc_sorted = data_inc.sort_values(by = 'DAT_CST', ascending=True).reset_index(drop=True)
data_d0 = data_inc_sorted.iloc[465:][['name','DAT_CST','registration_time']]
data_d0.shape
data_d0

,name,DAT_CST,registration_time
465,004-004-04-21-41-0190,2021-09-13,12:05
466,001-001-01-21-11-0128,2021-09-13,12:26
467,003-003-03-21-31-0109,2021-09-13,13:04
468,004-004-04-21-41-0189,2021-09-13,10:52
469,004-004-04-21-41-0191,2021-09-13,13:25
470,004-004-04-21-41-0192,2021-09-13,14:04
471,002-002-02-21-23-0048,2021-09-13,11:14
472,001-001-01-21-11-0127,2021-09-13,9:42
473,002-002-02-21-23-0050,2021-09-13,18:18
474,003-003-03-21-31-0110,2021-09-15,18:55


In [1701]:
data_inc_sorted.drop([471,474,467,473,469],inplace=True)

In [1702]:
data_inc = data_inc_sorted

In [1703]:
# groupby = data.groupby('name',sort = True).std()
# groupby

In [1704]:
# groupby = groupby.sort_values(by = 'AGE', ascending=True)

In [1705]:
# g = groupby[groupby['AGE']>1]
# g

In [1706]:
variable = ['name','CENTRE','AGE','AGE_MOIS','SEXE','TYPE_VIS','BARCODE','DAT_CST','STUDY_COMPLETE']

In [1707]:
# def CheckID(x,y,string):
#     data_list = []
#     data_all = []
#     for row in list(x.index):
#         df1 = y[y[string]==row][variable]
# #         df1.style.set_properties(**{'background-color': col,
# #                            'color': 'black',
# #                            'border-color': 'white'})             # Ce code est pour afficher pour chaque epid les personnes 
#         data_list.append(df1)  
#     data_all = pd.concat(data_list,axis = 0)
#     return data_all

In [1708]:
# data_to_clean_0 = CheckID(g,data,'name')
# data_to_clean_0

In [1709]:
# data_to_clean_0.to_excel('Doublons_ID')

In [1710]:
name_counts = data.name.value_counts().to_frame()
name_counts

,name
001-001-01-21-14-0022,6
001-001-01-21-11-0069,6
001-001-01-21-12-0031,6
001-001-01-21-12-0001,5
003-003-03-21-31-0102,5
...,...
004-004-04-21-41-0004,1
001-001-01-21-14-0010,1
001-001-01-21-11-0044,1
001-001-01-21-14-0012,1


In [1711]:
ID_with_one_record = name_counts[name_counts.name == 1]
ID_with_one_record

ID_with_3_record = name_counts[name_counts.name == 3]
ID_with_3_record

ID_with_2_record = name_counts[name_counts.name == 2]
ID_with_2_record.shape

ID_with_upthan3_record = name_counts[name_counts.name > 3]
ID_with_upthan3_record.shape

ID_with_one_record.shape

(9, 1)

## On a 475 enrollements dont 470 retenus parmis les 470 il y'a 7 qui sont des doublons (meme ID pour deux personnes differents). Il y'a 404 patients avec 3 visites, 35 patients qui ont 2 visites seulement, 20 patients ayant plus de 3 visites et 9 avec une seule visite

In [1712]:
def CheckID(x,y,string): # x est le nom du dataset de frequence des epids, y est le dataset global et string est le nom de la colonne contenant les EPIDS 
    data_list = []      # Pour stocker les datasets dont tous les index on le meme epid
    for row in list(x.index):
        df1 = y[y[string]==row][variable]
#         df1.style.set_properties(**{'background-color': col,
#                            'color': 'black',
#                            'border-color': 'white'})             # Ce code est pour afficher pour chaque epid les personnes 
        data_list.append(df1)  
    data_all = pd.concat(data_list,axis = 0) # regroupe tous les datasets ayant le meme EPID 
    return data_all

In [1713]:
# data_to_clean = CheckID(name_counts,data,'name')
# data_to_clean.reset_index(inplace=True,drop=True)

data_to_clean1 = CheckID(name_counts,data,'name')
data_to_clean1.reset_index(inplace=True,drop=True)

In [1714]:
data_to_clean1.to_excel("Checking_incoherence_per_patients2.xlsx")

In [1715]:
code_bare_counts = data_inc.BARCODE.value_counts().to_frame()
code_bare_counts

,BARCODE
211096.0,2
211054.0,2
211199.0,1
211459.0,1
211434.0,1
...,...
211188.0,1
211123.0,1
211132.0,1
211124.0,1


In [1716]:
# liste1 = []
# liste2 = []
# liste3 = []
# liste4 = []
# liste5 = []
# liste6 = []
# liste6_1 = []
# liste7 = []
# liste7_1 = []
# liste1_1 = []
# liste8 = []
# print(data_nan.shape)
# for name in data_d0.name.unique():
#     if name in data_d3.name.unique():
#         liste1.append(name)
#     else:
#         liste1_1.append(name)
# print("----------------------Comparison btw D0 and D3-------------------------------------------")
       
# print(len(data_d0.name.unique()))
# print(liste1,len(liste1))
# print(liste1_1,len(liste1_1))

# for name in data_d0.name.unique():
#     if name in data_d7.name.unique():
#         liste2.append(name)
# print("------------------------Comparison btw D0 and D7------------------------------------------------")
# print(len(data_d0.name.unique()))
# print(len(liste2))

# print("------------------------Comparison btw D0 and UV------------------------------------------------")

# for name in data_d0.name.unique():
#     if name in data_uv.name.unique():
#         liste3.append(name)
        
# print(len(data_d0.name.unique()))
# print(len(liste3))

# print("------------------------Comparison btw D3 and UV------------------------------------------------")

# for name in data_uv.name.unique():
#     if name in data_d3.name.unique():
#         liste4.append(name)
        
# print(len(data_uv.name.unique()))
# print(len(liste4))

# print("------------------------Comparison btw D7 and UV------------------------------------------------")

# for name in data_uv.name.unique():
#     if name in data_d7.name.unique():
#         liste5.append(name)
        
# print(len(data_uv.name.unique()))
# print(len(liste5))

# print("------------------------D3 in UV if is in D7?------------------------------------------------")

# for name in liste4:
#     if name in data_d7.name.unique():
#         liste6.append(name)
#     else:
#         liste6_1.append(name)
        
# print(len(liste4))
# print(liste6,len(liste6))
# print(liste6_1,len(liste6_1))

# print("------------------------D7 in UV if is in D3?------------------------------------------------")

# for name in liste5:
#     if name in data_d3.name.unique():
#         liste7.append(name)
#     else:
#         liste7_1.append(name)
        
# print(len(liste5))
# print(liste7,len(liste7))
# print(liste7_1,len(liste7_1))

# for name in liste1_1:
#     if name in data_uv.name.unique():
#         liste8.append(name)
    
        
# print(liste8,len(liste8))
# # print(liste6,len(liste6))
# # print(liste6_1,len(liste6_1))

In [1717]:
# a = data_d0.loc[data_d0['name'].isin(liste6_1)].sort_values(by='name',ascending = True)
# b = data_uv.loc[data_uv['name'].isin(liste6_1)].sort_values(by='name',ascending = True)
# a.reset_index(inplace=True,drop=True)
# b.reset_index(inplace=True,drop=True)
# a_b = pd.concat([a,b],axis=1)
# a_b[['name','DAT_CST']]
# # print(a.shape)
# # print(b.shape)

In [1718]:
# print('Patients D0 to D7:',len(liste2)+len(liste6_1))

In [1719]:
# c = data_d0.loc[data_d0['name'].isin(liste7_1)].sort_values(by='name',ascending = True)
# d = data_uv.loc[data_uv['name'].isin(liste7_1)].sort_values(by='name',ascending = True)
# c.reset_index(inplace=True,drop=True)
# d.reset_index(inplace=True,drop=True)
# c_d = pd.concat([c,d],axis=1)
# c_d[['name','DAT_CST']]

In [1720]:
# print('Patients D0 to D3:',len(liste1)+len(liste7_1))

In [1721]:
# prlvt_bacterio = pd.read_excel("PROJET FIND.xlsx")
# prlvt_bacterio

In [1722]:
# prlvt_bacterio.AGE.unique()

In [1723]:
# prlvt_bacterio.AGE.isnull().sum()

In [1724]:
# prlvt_bacterio.AGE = prlvt_bacterio.AGE.astype(float)
# prlvt_bacterio_find = prlvt_bacterio[prlvt_bacterio.AGE < 15]
# prlvt_bacterio_find.reset_index(inplace=True,drop=True)
# print(prlvt_bacterio_find.shape)
# prlvt_bacterio_find[['AGE','CENTRE']]

In [1725]:
# # duplicated ID at the entry
# data_inc_vc = data_inc.name.value_counts().to_frame()
# data_inc_vc[data_unc_vc.name > 1]

In [1726]:
# # duplicated ID at D3
# data_d3_vc = data_d3.name.value_counts().to_frame()
# data_d3_vc[data_d3_vc.name > 1]

In [1727]:
# # duplicated ID at D7
# data_d7_vc = data_d7.name.value_counts().to_frame()
# data_d7_vc[data_d7_vc.name > 1]

In [1728]:
# # data_d3.loc[liste4,'name']
# a = data_d3.loc[data_d3['name'].isin(liste4)].sort_values(by='name',ascending = True)
# b= data_uv.loc[data_uv['name'].isin(liste4)].sort_values(by='name',ascending = True)
# a.reset_index(inplace=True,drop=True)
# b.reset_index(inplace=True,drop=True)
# a_b = pd.concat([a,b],axis=1)
# a_b[['name','DAT_CST']]

In [1729]:
# l= []
# m = []
# for name in liste4:
#     d_uv = data_uv[data_uv.name == name][['name','DAT_CST']]
#     d3 = data_d3[data_d3.name == name][['name','DAT_CST']]
#     l.append(d_uv)
#     l_total = pd.concat(l,axis=0)
#     m.append(d3)
#     m_total = pd.concat(m,axis=0)
# m_total
# for i in range(len(liste4)):
#     if data_d3.DAT_CST.reset_index(inplace=True,drop=True).iloc[i] == data_uv.DAT_CST.reset_index(inplace=True,drop=True).iloc[i]:
#         print((data_uv.name,data_uv.DAT_CST))

In [1730]:
df

,Site,Lt,Ln,Nbre_inclus
0,Fatick,14.35810,-16.58580,111
1,Kedougou,12.55983,-12.19188,108
2,Mbour,14.41170,-16.96580,124
3,Tambacounda,13.77400,-13.66600,132


In [1731]:
# solar = 'https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/slate/bootstrap.min.css'
minty = 'https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/minty/bootstrap.min.css'
# journal = 'https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/journal/bootstrap.min.css'
# lumen = 'https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/lumen/bootstrap.min.css'

# app = dash.Dash(external_stylesheets=[dbc.themes.SOLAR])
app = dash.Dash(external_stylesheets=[minty])
app.title = 'FIND CDSA ENROLLMENT STATUS'
# 

colors = {'text':'#456FBV','plot':'#D3D3D3','paper':'#F5FFFA'}
color_palettes = ["#94d0cc","#eec4c4","#ff7b54","#d1d9d9"]

In [1732]:
data_mbour = data_inc[data_inc.CENTRE == "Mbour"]
data_fatick = data_inc[data_inc.CENTRE == 'Fatick']
data_kedougou = data_inc[data_inc.CENTRE == 'Kedougou']
data_tamba = data_inc[data_inc.CENTRE == 'Tambacounda']

# -------------------------------------------------------------------------------------------------------------------------


enr_day = data_inc.set_index('DAT_CST')
date  = enr_day.index.value_counts().to_frame()
date.reset_index(inplace=True,drop=False)
# date.drop('level_0',axis=1,inplace=True)
date.rename(columns={'index':'Dates','DAT_CST':'Counts'},inplace=True)
date.sort_values(by= 'Dates',ascending = True, inplace=True)

fievre = data_inc.FIEVRE_YN.value_counts().to_frame()
fievre.reset_index(inplace=True,drop=False)
fievre.rename(columns={'index':'Fever','FIEVRE_YN':'Counts'},inplace=True)

diarrhea = data_inc.DIARRHEES.value_counts().to_frame()
diarrhea.reset_index(inplace=True,drop=False)
diarrhea.rename(columns={'index':'Diarrhea','DIARRHEES':'Counts'},inplace=True)

study_complete = data_inc.STUDY_COMPLETE.value_counts().to_frame()
study_complete.reset_index(inplace=True,drop=False)
study_complete.rename(columns={'index':'Study complete','STUDY_COMPLETE':'Counts'},inplace=True)

data_ei = data_ei['owner_name'].value_counts().to_frame()
data_ei.reset_index(inplace=True,drop=False)
data_ei.rename(columns={'index':'Region','owner_name':'Counts'},inplace=True)


# -------------------------------------------datasets----------------------------------------------------------------------
width = 1500 
# height= 700
#----------------------------------------map---------------------------------------------------------------------------------
fig1 = px.scatter_mapbox(df, lat="Lt", lon="Ln", color="Site", size="Nbre_inclus",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                  mapbox_style="carto-positron",width=width)
fig1.update_layout(
    title_text="Enrollment per region", title_x=0.5,title_y=0.9)
#----------------------------------------Enrollment per arm fig-----------------------------------------------------------

# fig2 = px.bar(arm, x=arm.ARM, 
#               y=arm.Counts,text = arm.Counts)
# fig2.update_layout(
#     title_text="Enrollment per arm", title_x=0.5,title_y=0.9)

#----------------------------------------Enrollment per day fig-----------------------------------------------------------
# weight = 
# height=

fig3 = px.line(date, x=date.Dates, y=date.Counts)
fig3.update_layout(
    title_text="Enrollment per day", title_x=0.5,title_y=0.9)  

# fig4 = go.Figure(data=[go.Pie(labels=data_inc.AGE_CAT.value_counts().to_frame().index, 
#                               values=data_inc.AGE_CAT.value_counts().to_frame().AGE_CAT, textinfo='label+percent',
#                              insidetextorientation='radial',hole=.4
#                             )])

# fig5 = go.Figure(data=[go.Pie(labels=data_inc.SEXE.value_counts().to_frame().index, 
#                               values=data_inc.SEXE.value_counts().to_frame().SEXE, textinfo='label+percent',
#                              insidetextorientation='radial',hole=.4
#                             )])

# fig6 = go.Figure(data=[go.Pie(labels=data_inc.FIEVRE_YN.value_counts().to_frame().index, 
#                               values=data_inc.FIEVRE_YN.value_counts().to_frame().FIEVRE_YN, textinfo='label+percent',
#                              insidetextorientation='radial',hole=.4
#                             )])

# fig6 = px.bar(fievre, x=fievre.Fever, 
#               y=fievre.Counts,
#               text = fievre.Counts)

# fig6.update_layout(
#     title_text="Fever case count", title_x=0.5,title_y=0.9)

# fig7 = px.bar(diarrhea, x=diarrhea.Diarrhea, 
#               y=diarrhea.Counts,
#               text = diarrhea.Counts)

# fig7.update_layout(
#     title_text="Diarrhea case count", title_x=0.5,title_y=0.9)

# fig8 = px.bar(study_complete, x=study_complete['Study complete'], 
#               y=study_complete.Counts,
#               text = study_complete.Counts)

# fig8.update_layout(
#     title_text="Study complete", title_x=0.5,title_y=0.9)


fig_ei = px.bar(data_ei, x= data_ei.Region, y=data_ei.Counts,
                text = data_ei.Counts)

fig_ei.update_layout(
    title_text="Adverse events count", title_x=0.5,title_y=0.9)
#-----------------------------------------first Card dropdown---------------------------------------------------------------
dropdownlist1 = dbc.Card([ 
    
    dbc.FormGroup([
                
                dbc.Label("Months"),
                dcc.Dropdown(
                    id="months",
                    options=[
                        {"label": col, "value": col} for col in date.Dates.dt.month.apply(lambda x: calendar.month_name[x]).unique()
                    ])
               ]),
    

    dbc.FormGroup([
        
        dbc.Label('Date range'),
        
        dcc.DatePickerRange(
        id = 'date_range',
        min_date_allowed=data_inc.DAT_CST.min().date(),
        max_date_allowed=data_inc.DAT_CST.max().date(),
        start_date=data_inc.DAT_CST.min().date(),
        end_date=data_inc.DAT_CST.max().date(),
        )
    ]),
    
    dbc.FormGroup([
        
        dbc.Label("ARM"),
        dcc.Dropdown(
        id = 'bras',
        options=[{'label':col,'value':col} for col in data_inc.ARM.unique()]
        )
    ])
]),


dropdownlist2 = dbc.Card([ 
    
     dbc.FormGroup([
        
        dbc.Label("Region"),
        dcc.Dropdown(
        id = 'region2',
        options=[{'label':col,'value':col} for col in data_inc.CENTRE.unique()]
        )
])
])

dropdownlist3 = dbc.Card([ 
    
     dbc.FormGroup([
        
        dbc.Label("Region"),
        dcc.Dropdown(
        id = 'region3',
        options=[{'label':col,'value':col} for col in data_inc.CENTRE.unique()]
        )
])
])
#----------------------------------------Components of the first tap-----------------------------------------------------------
# first_comp = dcc.Graph(id = 'map', figure=fig1,style={'color':'black'})


second_comp=dbc.Row([
                
            dbc.Col(dropdownlist1,md=4),
            dbc.Col(dcc.Graph(id = 'line_date')),
            dbc.Row([   
            dbc.Col(dcc.Graph(id = 'bar_arm_d0'),md=5),
            dbc.Col(dcc.Graph(id = 'bar_arm_d3'),md=5),
            dbc.Col(dcc.Graph(id = 'bar_arm_d7'),md=5),
            dbc.Col(dcc.Graph(id = 'bar_arm_uv'),md=5)
            ])
])

# ------------------------------------------Cards for RDT-------------------------------------------------------------------
card_content1 = [
    dbc.CardHeader(html.H6("Arbo suspiscion/Dengue RDTs")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_inc.ARBOVIROSE.value_counts().loc['Yes'])+'/'+str(data_inc.TDR_DENGUE_FAIT.value_counts().loc['Yes']),
            className="card-rdtdengue"),
            html.P(
                "Mbour: "+str(data_mbour.ARBOVIROSE.value_counts().loc['Yes'])+'/'+str(data_mbour.TDR_DENGUE_FAIT.value_counts().loc['Yes']),
                className="card-text1",
            ),
        html.P(
                "Tamba: "+str(data_tamba.ARBOVIROSE.value_counts().loc['Yes'])+'/'+str(data_tamba.TDR_DENGUE_FAIT.value_counts().loc['Yes']),
                className="card-text4",
         ), 
        ]
    ),
]

card_content2 = [
    dbc.CardHeader(html.H6("Sore throat suspiscion/Strepto A RDTs")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_inc.MGORGE.value_counts().loc['Yes'])+'/'+str(data_inc.TDR_STREPTO_A_FAIT.value_counts().loc['Yes']),
            className="card-rdtstrepto"),
        html.P(
                "Mbour: "+str(data_mbour.MGORGE.value_counts().loc['Yes'])+'/'+str(data_mbour.TDR_STREPTO_A_FAIT.value_counts().loc['Yes']),
                className="card-text1",
            ),
        html.P(
                "Tamba: "+str(data_tamba.MGORGE.value_counts().loc['Yes'])+'/'+str(data_tamba.TDR_STREPTO_A_FAIT.value_counts().loc['Yes']),
                className="card-text2",
         ), 
        ]
    ),
]

# card_content3 = [
#     dbc.CardHeader("Arbo suspiscion/Strepto A RDTs"),
#     dbc.CardBody(
#         [
#             html.H3(str(data_inc.MGORGE.value_counts().loc['Yes'])+'/'+str(data_inc.TDR_STREPTO_A_FAIT.value_counts().loc['Yes']),
#             className="card-rdtstrepto")
# #             html.P(
# #                 "This is some card content that we'll reuse",
# #                 className="card-text",
# #             ),
#         ]
#     ),
# ]

card_content4 = [
    dbc.CardHeader(html.H6("Arbo suspiscion/Blood sample collection")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_inc.ARBOVIROSE.value_counts().loc['Yes'])+'/'+str(data_inc.SANGUIN.value_counts().loc['Yes']),
            className="card-bloodsamplecollection"),
            html.P(
                "Mbour: "+str(data_mbour.ARBOVIROSE.value_counts().loc['Yes'])+'/'+str(data_mbour.SANGUIN.value_counts().loc['Yes']),
                className="card-text1",
            ),
            html.P(
                "Fatick: "+str(data_fatick.ARBOVIROSE.value_counts().loc['Yes'])+'/'+str(data_fatick.SANGUIN.value_counts().loc['Yes']),
                className="card-text2",
            ),
            html.P(
                "Kedougou: "+str(data_kedougou.ARBOVIROSE.value_counts().loc['Yes'])+'/'+str(data_kedougou.SANGUIN.value_counts().loc['Yes']),
                className="card-text3",
            ),
            html.P(
                "Tambacounda: "+str(data_tamba.ARBOVIROSE.value_counts().loc['Yes'])+'/'+str(data_tamba.SANGUIN.value_counts().loc['Yes']),
                className="card-text4",
            ),
        ]
    ),
]
card_content5 = [
    dbc.CardHeader(html.H6("Diarrhea suspiscion/Stool sample collection")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_inc.DIARRHEES.value_counts().loc['Diarrhea'])+'/'+str(data_inc.SELLES.value_counts().loc['Yes']),
            className="card-stoolsamplecollection"),
        html.P(
                "Mbour: "+str(data_mbour.DIARRHEES.value_counts().loc['Diarrhea'])+'/'+str(data_mbour.SELLES.value_counts().loc['Yes']),
                className="card-text1",
            ),
        html.P(
                "Fatick: "+str(data_fatick.DIARRHEES.value_counts().loc['Diarrhea'])+'/'+str(data_fatick.SELLES.value_counts().loc['Yes']),
                className="card-text2",
            ),
        html.P(
                "Kedougou: "+str(data_kedougou.DIARRHEES.value_counts().loc['Diarrhea'])+'/'+str(data_kedougou.SELLES.value_counts().loc['Yes']),
                className="card-text3",
            ),
        html.P(
                "Tambacounda: "+str(data_tamba.DIARRHEES.value_counts().loc['Diarrhea'])+'/'+str(data_tamba.SELLES.value_counts().loc['Yes']),
                className="card-text4",
            ),
         
        ]
    ),
]

card_content6 = [
    dbc.CardHeader(html.H6("Flu synd/Naso sample collection")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_inc.SYND_GRIPPAL.value_counts().loc['Yes'])+'/'+str(data_inc.ECOUV_NASAL.value_counts().loc['Yes']),
            className="card-nasosamplecollection"),
        html.P(
                "Mbour: "+str(data_mbour.SYND_GRIPPAL.value_counts().loc['Yes'])+'/'+str(data_mbour.ECOUV_NASAL.value_counts().loc['Yes']),
                className="card-text1",
            ),
      html.P(
                "Fatick: "+str(data_fatick.SYND_GRIPPAL.value_counts().loc['Yes'])+'/'+str(data_fatick.ECOUV_NASAL.value_counts().loc['Yes']),
                className="card-text2",
            ),
      html.P(
                "Kedougou: "+str(data_kedougou.SYND_GRIPPAL.value_counts().loc['Yes'])+'/'+str(data_kedougou.ECOUV_NASAL.value_counts().loc['Yes']),
                className="card-text3",
            ),
      html.P(
                "Tambacounda: "+str(data_tamba.SYND_GRIPPAL.value_counts().loc['Yes'])+'/'+str(data_tamba.ECOUV_NASAL.value_counts().loc['Yes']),
                className="card-text4",
            ),
        ]
    ),
]


cards = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content1, color="primary", inverse=True)),
                dbc.Col(
                    dbc.Card(card_content2, color="secondary", inverse=True)
                ),
#                 dbc.Col(dbc.Card(card_content3, color="info", inverse=True)),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_content4, color="info", inverse=True)),
                dbc.Col(dbc.Card(card_content5, color="warning", inverse=True)),
                dbc.Col(dbc.Card(card_content6, color="danger", inverse=True)),
            ],
            className="mb-4",
        ),
#         dbc.Row(
#             [
#                 dbc.Col(dbc.Card(card_content, color="light")),
#                 dbc.Col(dbc.Card(card_content, color="dark", inverse=True)),
#             ]
#         ),
    ]
)

# -----------------------------------------------------------Cards dm--------------------------------------------------------
card_contentA = [
    dbc.CardHeader(html.H6("Corrected data")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_dcf['Commentaires cliniciens'].value_counts().loc['donnée corrigée']),
            className="card-donneecorrige"),
            html.P(
                "Mbour: "+str(data_dcf[data_dcf.Site == 'Mbour']['Commentaires cliniciens'].value_counts().loc['donnée corrigée']),
                className="card-donneecorrige1",
            ),
#             html.P(
#                 "Fatick: "+str(data_dcf[data_dcf.Site == 'Fatick']['Commentaires cliniciens'].value_counts().loc['donnée corrigée']),
#                 className="card-donneecorrige2",
#                  ), 
            html.P(
                "Kedougou: "+str(data_dcf[data_dcf.Site == 'Kedougou']['Commentaires cliniciens'].value_counts().loc['donnée corrigée']),
                className="card-donneecorrige3",
                 ),
            html.P(
                "Tamba: "+str(data_dcf[data_dcf.Site == 'Tambacounda']['Commentaires cliniciens'].value_counts().loc['donnée corrigée']),
                className="card-donneecorrige4",
             ),
        ]
    ),
]

card_contentB = [
    dbc.CardHeader(html.H6("Confirmed data")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_dcf['Commentaires cliniciens'].value_counts().loc['confirmé']),
            className="card-confirme"),
        html.P(
                "Mbour: "+str(data_dcf[data_dcf.Site == 'Mbour']['Commentaires cliniciens'].value_counts().loc['confirmé']),
                className="card-confirme1",
            ),
#         html.P(
#                 "Fatick: "+str(data_dcf[data_dcf.Site == 'Fatick']['Commentaires cliniciens'].value_counts().loc['confirmé']),
#                 className="card-confirme2",
#          ), 
        html.P(
                "Kedougou: "+str(data_dcf[data_dcf.Site == 'Kedougou']['Commentaires cliniciens'].value_counts().loc['confirmé']),
                className="card-confirme3",
         ),
        html.P(
                "Tamba: "+str(data_dcf[data_dcf.Site == 'Tambacounda']['Commentaires cliniciens'].value_counts().loc['confirmé']),
                className="card-confirme4",
         ),
        ]
    ),
]



card_contentC = [
    dbc.CardHeader(html.H6("Pending")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_dcf['Commentaires cliniciens'].value_counts().loc['en cours de vérification']),
            className="card-pending"),
            html.P(
                "Mbour: "+str(data_dcf[data_dcf.Site == 'Mbour']['Commentaires cliniciens'].value_counts().loc['en cours de vérification']),
                className="card-pending1",
            ),
#             html.P(
#                 "Fatick: "+str(data_dcf[data_dcf.Site == 'Fatick']['Commentaires cliniciens'].value_counts().loc['en cours de vérification']),
#                 className="card-pending2",
#             ),
            html.P(
                "Kedougou: "+str(data_dcf[data_dcf.Site == 'Kedougou']['Commentaires cliniciens'].value_counts().loc['en cours de vérification']),
                className="card-pending3",
            ),
#             html.P(
#                 "Tambacounda: "+str(data_dcf[data_dcf.Site == 'Tambacounda']['Commentaires cliniciens'].value_counts().loc['en cours de vérification']),
#                 className="card-pending4",
#             ),          
        ]
    ),
]

card_contentD = [
    dbc.CardHeader(html.H6("Total")),
    dbc.CardBody(
        [
            html.H4("Total: "+str(data_dcf.shape[0]),
            className="card-total"),
        html.P(
                "Mbour: "+str(data_dcf[data_dcf.Site == 'Mbour'].shape[0]),
                className="card-total1",
            ),
#         html.P(
#                 "Fatick: "+str(data_dcf[data_dcf.Site == 'Fatick']['Commentaires cliniciens'].value_counts().loc['confirmé']),
#                 className="card-confirme2",
#          ), 
        html.P(
                "Kedougou: "+str(data_dcf[data_dcf.Site == 'Kedougou'].shape[0]),
                className="card-total2",
         ),
        html.P(
                "Tamba: "+str(data_dcf[data_dcf.Site == 'Tambacounda'].shape[0]),
                className="card-total3",
        )
    ]
         
    ),
        
    
]

cards1 = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(dbc.Card(card_contentD, color="warning", inverse=True)),
                dbc.Col(dbc.Card(card_contentA, color="primary", inverse=True)),
                dbc.Col(
                    dbc.Card(card_contentB, color="secondary", inverse=True)
                ),
                dbc.Col(dbc.Card(card_contentC, color="info", inverse=True)),
            ],
            className="mb-4",
        ),
#         dbc.Row(
#             [
#                 dbc.Col(dbc.Card(card_content4, color="info", inverse=True)),
# #                 dbc.Col(dbc.Card(card_content5, color="warning", inverse=True)),
# #                 dbc.Col(dbc.Card(card_content6, color="danger", inverse=True)),
#             ],
#             className="mb-4",
#         ),
#         dbc.Row(
#             [
#                 dbc.Col(dbc.Card(card_content, color="light")),
#                 dbc.Col(dbc.Card(card_content, color="dark", inverse=True)),
#             ]
#         ),
    ]
)
table = dash_table.DataTable(
    id='tabl',
    columns=[{"name": i, "id": i,"deletable": True, "selectable": True} for i in data_to_clean.columns],
    data=data_to_clean.to_dict('records'),
    editable=True,
    filter_action="native",
    sort_action="native",
    sort_mode="multi",
    column_selectable="single",
    row_selectable="multi",
    row_deletable=True,
    selected_columns=[],
    selected_rows=[],
    page_action="native",
    page_current= 0,
    page_size= 10,
)
# ----------------------------------------App layout---------------------------------------------------------------------------

app.layout = dbc.Container([
            
    
#             dcc.Store(id='store',data={'fig1':fig1,'fig2':fig2,'fig3':fig3}),
            html.H3(children='FIND CDSA ENROLLMENT STATUS',style={'textAlign':'left','color':'salmon'
                                                                   }),
             
    
            html.Hr(style={"color": "salmon"}),
    
#             html.Div(children='This dashboard gives a global view of the FIND CDSA project enrollments',style={
#                     'textAlign' : 'center', 'color':colors['text']}),
            
            dbc.Tabs(id="tabs",
            active_tab="ES",children=
            [
                dbc.Tab(label="Enrollment status", tab_id="ES"),
                dbc.Tab(label="General statistics", tab_id="GS"),
                dbc.Tab(label="Suspiscion/RDTs-Sample collection", tab_id="SC"),
                dbc.Tab(label="Adverse Event (AE)", tab_id="AE"),
                dbc.Tab(label="Study complete", tab_id="SCm"),
                dbc.Tab(label="General overview of prescrited antiobiotics", tab_id="GOA"),
                dbc.Tab(label="Data management", tab_id="DM"),

            ],
            ),
          html.Div(id="tab-content"),
#           dcc.Graph(id = 'map', figure=fig1),
#           html.Div(children = [second_comp]),
          
#           html.Div(children=cards)
])



# G1 = dcc.Graph(id = 'map', figure=fig1),
# G2 = html.Div(children = [second_comp]),
# G3 = html.Div(children=cards)

@app.callback(Output("tab-content","children"),
              [Input("tabs", "active_tab")])
  
def render_tab_content(tab):
    if tab is not None:
        if tab == "ES":
            return html.Div([dcc.Graph(id = 'map',figure=fig1),
                            second_comp])
#           dcc.Graph(id = 'map',figure=data['fig1'])
        elif tab == "SC":
            return html.Div([
                html.Br(),
                cards
                            ])
        elif tab == "GS":
            return html.Div([
                html.Br(),
                dbc.Col([
                dbc.Col(dropdownlist3,md=2),
                dbc.Row([   
                dbc.Col(dcc.Graph(id = 'age')),
                dbc.Col(dcc.Graph(id = 'sexe'))
                ]),
                dbc.Row([
                dbc.Col(dcc.Graph(id = 'fievre')),
                dbc.Col(dcc.Graph(id = 'diarrhea'))
                ])
                ])            
            ])
              
        
        elif tab == "AE":
            return dcc.Graph(id='ei',figure=fig_ei)
        elif tab == "GOA":
            return
        elif tab == "SCm":
            return html.Div([
                html.Br(),
                dbc.Row([
                
                dbc.Col(dropdownlist2, width=3),
                dbc.Col(dcc.Graph(id='studycomplete'))
                
            ])
        ])
        elif tab == "DM":
            return html.Div([
                html.Br(),
                cards1,
                html.Br(),
                table 
            ])
    return "No tab selected"

@app.callback(Output('studycomplete','figure'),[Input('region2','value')])

def StudyComplete(centre):
    if centre is not None:
        df1 = data_inc[data_inc.CENTRE == centre]['STUDY_COMPLETE'].value_counts().to_frame()
        df1.reset_index(inplace=True,drop=False)
        df1.rename(columns={'index':'Study complete','STUDY_COMPLETE':'Counts'},inplace=True)
        fig = px.bar(df1, x=df1['Study complete'], 
                          y=df1.Counts,
                          text = df1.Counts)

    #             fig.update_layout(title_text="Study complete", title_x=0.5,title_y=0.9)
        return fig
    else:
        df1_1 = data_inc['STUDY_COMPLETE'].value_counts().to_frame()
        df1_1.reset_index(inplace=True,drop=False)
        df1_1.rename(columns={'index':'Study complete','STUDY_COMPLETE':'Counts'},inplace=True)
        fig1_1 = px.bar(df1_1 , x=df1_1['Study complete'], 
                          y=df1_1.Counts,
                          text = df1_1.Counts)
        return fig1_1
        
        
@app.callback([Output('age','figure'),Output('sexe','figure'),Output('fievre','figure'),Output('diarrhea','figure')],
              [Input('region3','value')])        
        
def Generalstatistics(centre):
    if centre is not None:
        df2 = data_inc[data_inc.CENTRE == centre]['AGE_CAT'].value_counts().to_frame()
        df2.reset_index(inplace=True,drop=False)
        df2.rename(columns={'index':'Age','AGE_CAT':'Counts'},inplace=True)
        fig2 = go.Figure(data=[go.Pie(labels=df2.Age, 
                                  values=df2.Counts, textinfo='label+percent',
                                 insidetextorientation='radial',hole=.4
                                )])
        df3 = data_inc[data_inc.CENTRE == centre]['SEXE'].value_counts().to_frame()
        df3.reset_index(inplace=True,drop=False)
        df3.rename(columns={'index':'Sex','SEXE':'Counts'},inplace=True)
        fig3 = go.Figure(data=[go.Pie(labels=df3.Sex, 
                                  values=df3.Counts, textinfo='label+percent',
                                 insidetextorientation='radial',hole=.4
                                )])
        df4 = data_inc[data_inc.CENTRE == centre]['FIEVRE_YN'].value_counts().to_frame()
        df4.reset_index(inplace=True,drop=False)
        df4.rename(columns={'index':'Fever','FIEVRE_YN':'Counts'},inplace=True)
        fig4 = px.bar(df4, x=df4.Fever, 
                  y=df4.Counts,
                  text = df4.Counts)

        df5 = data_inc[data_inc.CENTRE == centre]['DIARRHEES'].value_counts().to_frame()
        df5.reset_index(inplace=True,drop=False)
        df5.rename(columns={'index':'Diarrhea','DIARRHEES':'Counts'},inplace=True)
        fig5 = px.bar(df5, x=df5.Diarrhea, 
                  y=df5.Counts,
                  text = df5.Counts)

        return fig2,fig3,fig4,fig5
    
    else:
        df2_2 = data_inc['AGE_CAT'].value_counts().to_frame()
        df2_2.reset_index(inplace=True,drop=False)
        df2_2.rename(columns={'index':'Age','AGE_CAT':'Counts'},inplace=True)
        fig2_2 = go.Figure(data=[go.Pie(labels=df2_2.Age, 
                                  values=df2_2.Counts, textinfo='label+percent',
                                 insidetextorientation='radial',hole=.4
                                )])
        df3_3 = data_inc['SEXE'].value_counts().to_frame()
        df3_3.reset_index(inplace=True,drop=False)
        df3_3.rename(columns={'index':'Sex','SEXE':'Counts'},inplace=True)
        fig3_3 = go.Figure(data=[go.Pie(labels=df3_3.Sex, 
                                  values=df3_3.Counts, textinfo='label+percent',
                                 insidetextorientation='radial',hole=.4
                                )])
        df4_4 = data_inc['FIEVRE_YN'].value_counts().to_frame()
        df4_4.reset_index(inplace=True,drop=False)
        df4_4.rename(columns={'index':'Fever','FIEVRE_YN':'Counts'},inplace=True)
        fig4_4 = px.bar(df4_4, x=df4_4.Fever, 
                  y= df4_4.Counts,
                  text = df4_4.Counts)

        df5_5 = data_inc['DIARRHEES'].value_counts().to_frame()
        df5_5.reset_index(inplace=True,drop=False)
        df5_5.rename(columns={'index':'Diarrhea','DIARRHEES':'Counts'},inplace=True)
        fig5_5 = px.bar(df5_5, x=df5_5.Diarrhea, 
                  y=df5_5.Counts,
                  text = df5_5.Counts)

        return fig2_2,fig3_3,fig4_4,fig5_5
        

@app.callback([Output('bar_arm_d0','figure'),Output('bar_arm_d3','figure'),
               Output('bar_arm_d7','figure'),Output('bar_arm_uv','figure')],
             [Input('bras','value')])

def EnrollmentStatus1(bras):
    
#     enr_day = data_inc.set_index('DAT_CST')
#     date  = enr_day.index.value_counts().to_frame()
#     date.reset_index(inplace=True,drop=False)
#     # date.drop('level_0',axis=1,inplace=True)
#     date.rename(columns={'index':'Dates','DAT_CST':'Counts'},inplace=True)
#     date.sort_values(by= 'Dates',ascending = True, inplace=True)
    
#     fig7 = px.line(date, x=date.Dates, y=date.Counts)
#     fig7.update_layout(
#     title_text="Enrollment per day", title_x=0.5,title_y=0.9)
    
    arm = data_inc[data_inc.ARM == bras]['ARM'].value_counts().to_frame()
    arm.reset_index(inplace=True,drop=False)
    # date.drop('level_0',axis=1,inplace=True)
    arm.rename(columns={'index':'ARM','ARM':'Counts'},inplace=True)
    arm = arm.append({'ARM':'Total','Counts':arm.Counts.sum()},ignore_index = True)
    
    fig6 = px.bar(arm, x=arm.ARM, 
              y=arm.Counts,text = arm.Counts)
    fig6.update_layout(
    title_text="Enrollment per arm at D0", title_x=0.5,title_y=0.9)
    
    
    df_d3 = data_d3[data_d3.ARM == bras]['ARM'].value_counts().to_frame()
    df_d3.reset_index(inplace=True,drop=False)
    # date.drop('level_0',axis=1,inplace=True)
    df_d3.rename(columns={'index':'ARM','ARM':'Counts'},inplace=True)
    df_d3 = df_d3.append({'ARM':'Total','Counts':df_d3.Counts.sum()},ignore_index = True)
    
    fig8= px.bar(df_d3, x=df_d3.ARM, 
              y=df_d3.Counts,text = df_d3.Counts)
    fig8.update_layout(
    title_text="Enrollment per arm at D3", title_x=0.5,title_y=0.9)
    
    
    df_d7 = data_d7[data_d7.ARM == bras]['ARM'].value_counts().to_frame()
    df_d7.reset_index(inplace=True,drop=False)
    # date.drop('level_0',axis=1,inplace=True)
    df_d7.rename(columns={'index':'ARM','ARM':'Counts'},inplace=True)
    df_d7 = df_d7.append({'ARM':'Total','Counts':df_d7.Counts.sum()},ignore_index = True)
    
    fig9= px.bar(df_d7, x=df_d7.ARM, 
              y=df_d7.Counts,text = df_d7.Counts)
    fig9.update_layout(
    title_text="Enrollment per arm at D7", title_x=0.5,title_y=0.9)
    
    df_uv = data_uv[data_uv.ARM == bras]['ARM'].value_counts().to_frame()
    df_uv.reset_index(inplace=True,drop=False)
    # date.drop('level_0',axis=1,inplace=True)
    df_uv.rename(columns={'index':'ARM','ARM':'Counts'},inplace=True)
    df_uv = df_uv.append({'ARM':'Total','Counts':df_uv.Counts.sum()},ignore_index = True)
    
    fig10= px.bar(df_uv, x=df_uv.ARM, 
              y=df_uv.Counts,text = df_uv.Counts)
    fig10.update_layout(
    title_text="Enrollment per arm at UV", title_x=0.5,title_y=0.9)
    
    
#     arm_D0 = data_inc.ARM.value_counts().to_frame()
#     arm_D0.reset_index(inplace=True,drop=False)
#     # date.drop('level_0',axis=1,inplace=True)
#     arm_D0.rename(columns={'index':'ARM','ARM':'Counts'},inplace=True)
#     arm_D0 = arm_D0.append({'ARM':'Total','Counts':arm_D0.Counts.sum()},ignore_index = True)
    
#     fig11 = px.bar(arm_D0, x=arm_D0.ARM, 
#               y=arm_D0.Counts,text = arm_D0.Counts)
#     fig11.update_layout(
#     title_text="Enrollment per arm at D0", title_x=0.5,title_y=0.9)
    
#     arm_D3 = data_d3.ARM.value_counts().to_frame()
#     arm_D3.reset_index(inplace=True,drop=False)
#     # date.drop('level_0',axis=1,inplace=True)
#     arm_D3.rename(columns={'index':'ARM','ARM':'Counts'},inplace=True)
#     arm_D3 = arm_D3.append({'ARM':'Total','Counts':arm_D3.Counts.sum()},ignore_index = True)
    
#     fig12 = px.bar(arm_D3, x=arm_D3.ARM, 
#               y=arm_D3.Counts,text = arm_D3.Counts)
#     fig12.update_layout(
#     title_text="Enrollment per arm at D3", title_x=0.5,title_y=0.9)
    
#     arm_D7 = data_d7.ARM.value_counts().to_frame()
#     arm_D7.reset_index(inplace=True,drop=False)
#     # date.drop('level_0',axis=1,inplace=True)
#     arm_D7.rename(columns={'index':'ARM','ARM':'Counts'},inplace=True)
#     arm_D7 = arm_D7.append({'ARM':'Total','Counts':arm_D7.Counts.sum()},ignore_index = True)
    
#     fig13 = px.bar(arm_D7, x=arm_D7.ARM, 
#               y=arm_D7.Counts,text = arm_D7.Counts)
#     fig13.update_layout(
#     title_text="Enrollment per arm at D7", title_x=0.5,title_y=0.9)
    
    return fig6,fig8,fig9,fig10
#     return html.Div([dcc.Graph(dbc.Row([dbc.Col(id='dflt_arm0', figure = fig11),
#                                         dbc.Col(id='dflt_arm3', figure = fig12),
#                                         dbc.Col(id='dflt_arm7', figure = fig13)
   
# @app.callback([Output('line_date','figure')],
#              [Input('date_range','value')])

# # Input('months','value')
# def EnrollmentStatus2(dates):
#     date  = data_inc.DAT_CST.to_frame()
# #     date.reset_index(inplace=True,drop=False)
# #     # date.drop('level_0',axis=1,inplace=True)
# #     date.rename(columns={'index':'Dates','DAT_CST':'Counts'},inplace=True)
#     date.sort_values(by= 'DAT_CST',ascending = True, inplace=True)
    
#     fig11 = px.line(date, x=, y=date.value_counts().to_frame()Counts)
#     fig11.update_layout(
#     title_text="Enrollment per day", title_x=0.5,title_y=0.9)
    
#     return fig11


@app.callback(Output('line_date','figure'),
              [Input('date_range','start_date'),Input('date_range','end_date')]
)
def update_dates(start_date, end_date):
    date = data_inc[(data_inc.DAT_CST >= start_date) & (data_inc.DAT_CST >= end_date)]['DAT_CST'].value_counts().to_frame()
    date.reset_index(inplace=True,drop=False)
    # date.drop('level_0',axis=1,inplace=True)
    date.rename(columns={'index':'Date','DAT_CST':'Counts'},inplace=True)
    date.sort_values(by= 'Date',ascending = True, inplace=True)
    
#     mask = (
#         ()
#         & (data_inc.DAT_CST <= end_date)
#     )
#     filtered_data = data_inc.loc[mask, :]
    date_chart_figure = {
        "data": [
            {
                "x": date["Date"],
                "y": date["Counts"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {
                "text": "Enrollment per day",
                "x": 0.05,
                "xanchor": "left"
            },
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#87CEFA"],
        },
    }
    return date_chart_figure



@app.callback(
    Output('tabl', 'style_data_conditional'),
    Input('tabl', 'selected_columns')
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#87CEFA'
    } for i in selected_columns]



In [1733]:
# data_dcf[data_dcf.Site == 'Tambacounda']['Commentaires cliniciens'].value_counts()

In [1734]:
# print(data_fatick.STUDY_COMPLETE.value_counts(), '\n',data_mbour.STUDY_COMPLETE.value_counts(),'\n',
#       data_tamba.STUDY_COMPLETE.value_counts(),data_kedougou.STUDY_COMPLETE.value_counts())

In [1735]:
if __name__ == '__main__':
#     app.run_server(port = 4050)
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [1736]:
# san = data_inc[data_inc.SANGUIN == 'Yes'][['name','PRLVT','SANGUIN','enrolled_study','CENTRE','BARCODE','DAT_CST']]
# san
data_inc[data_inc.BARCODE == 211414][['name','AGE','PRLVT','SELLES','ECOUV_NASAL','SANGUIN','enrolled_study','CENTRE','BARCODE','DAT_CST']]
# data_inc[data_inc.name == '003-003-03-21-31-0109'][['name','AGE','PRLVT','SELLES','ECOUV_NASAL','SANGUIN','enrolled_study','CENTRE','BARCODE','DAT_CST']]
# data[data.NUMERO_TEL == '774123983'][['name','AGE','PRLVT','SELLES','ECOUV_NASAL','SANGUIN','enrolled_study','CENTRE','BARCODE','DAT_CST']]

# san[san.CENTRE == 'Kedougou'][['name','CENTRE','enrolled_study','BARCODE','DAT_CST','SANGUIN']]

,name,AGE,PRLVT,SELLES,ECOUV_NASAL,SANGUIN,enrolled_study,CENTRE,BARCODE,DAT_CST
221,004-004-04-21-41-0078,3.0,---,Yes,---,---,Y,Tambacounda,211414.0,2021-07-06


In [1737]:
data.NUMERO_TEL.dtype

dtype('O')

In [1738]:
# data_arbo = pd.read_excel('arbo_extaction.xlsx')
# data_arbo
# data_rdt = data_inc[(data_inc.TDR_DENGUE_FAIT == 'Yes') | (data_inc.TDR_STREPTO_A_FAIT == 'Yes')][['name','TDR_DENGUE_FAIT','TDR_STREPTO_A_FAIT','CENTRE','DAT_CST','HEURE_DBT_TDR_DENGUE','HEURE_FIN_TDR_DENGUE','HEURE_DBT_TDR_STREPTOA','HEURE_FIN_TDR_STREPTOA','link_hq_photo_tdr_dengue','link_hq_photo_tdr_streptoa']]
# data_rdt.reset_index(inplace=True,drop=True)
# data_rdt.to_excel('donneesTDRs2.xlsx')